In [15]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import shfl
import torch
import copy
import numpy as np
from imutils import paths
import cv2

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

In [16]:
args = {"data_path":"../data/COVIDGR1.0-Segmentadas", 
        "csv_path": "../partitions/partition1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 100,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
def classifier_builder():
    return ClassifierModel(batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

In [18]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

LabelBinarizer()

In [47]:
path = "../data/COVIDGR1.0-TransformadasSplit"
train_path = os.path.join(path, "train")
test_path = os.path.join(path, "test")
height = width = 224


t_train_data = []
t_train_labels = []
t_test_data = []
t_test_labels = []
        
train_files = list(paths.list_images(train_path))
        
for image_path in train_files:
    label = image_path.split(os.path.sep)[-2]

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if height != None and width != None:
        image = cv2.resize(image, (height, width))
            
    t_train_data.append(image)
    t_train_labels.append(label)
            
test_files = list(paths.list_images(test_path))
        
for image_path in test_files:
    label = image_path.split(os.path.sep)[-2]

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if height != None and width != None:
        image = cv2.resize(image, (height, width))
            
    t_test_data.append(image)
    t_test_labels.append(label)
    
    
t_train_data = np.array(t_train_data)
t_train_labels = np.array(t_train_labels)
t_test_data = np.array(t_test_data)
t_test_labels = np.array(t_test_labels)
        
t_train_labels = lb2.transform(t_train_labels)
t_test_labels = lb2.transform(t_test_labels)

In [58]:
from ClassifierModel import ClassifierModel

classifier_model = classifier_builder()
classifier_model.train(t_train_data, t_train_labels)

Epoch 1/100
136/136 [==============================] - 29s 216ms/step - loss: 0.6622 - categorical_accuracy: 0.7168 - val_loss: 3.0349 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
136/136 [==============================] - 10s 75ms/step - loss: 0.3737 - categorical_accuracy: 0.8303 - val_loss: 1.1796 - val_categorical_accuracy: 0.4118
Epoch 3/100
136/136 [==============================] - 10s 74ms/step - loss: 0.2054 - categorical_accuracy: 0.9280 - val_loss: 1.0764 - val_categorical_accuracy: 0.5735
Epoch 4/100
136/136 [==============================] - 10s 74ms/step - loss: 0.1388 - categorical_accuracy: 0.9437 - val_loss: 3.9094 - val_categorical_accuracy: 0.0551
Epoch 5/100
136/136 [==============================] - 10s 73ms/step - loss: 0.1316 - categorical_accuracy: 0.9613 - val_loss: 4.8847 - val_categorical_accuracy: 0.0662
Epoch 6/100
136/136 [==============================] - 10s 74ms/step - loss: 0.0961 - categorical_accuracy: 0.9705 - val_loss: 1.9238 - val_categorica

In [59]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report


model = classifier_model._model

def get_classification_report(test_files, dict_labels):
    true_labels = []
    preds = []
    no_concuerda = 0
    preds_4 = []
    true_labels_4 = []
    tabla_preds = np.empty((len(test_files)//2, 3), dtype = '<U50')
        
    for i in range(len(test_files)//2):
        image_path = test_files[0]
        curr_image = image_path.split(os.path.sep)[-1]
        name = curr_image.split('_')[0]
        transf = curr_image.split('_')[1].split('T18')[1]
        label = curr_image.split('_')[1].split('T18')[0]
                    
        true_labels.append(label)
        true_labels_4.append(dict_labels[label + "TP"])
        true_labels_4.append(dict_labels[label + "TN"])
        tabla_preds[i,0] = name
            
        #image = cv2.imread(image_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, (256, 256))
            
        #x = sample_loader(image)
        #tp = G_dict['P'](x)
        #tp = tp[0].cpu().detach().numpy()
        #tp = np.moveaxis(tp, 0, -1)
        
        tp_path = path + '/test/' + label + 'TP' + '/' + name + '_' + label + 'T18P.png'  
        tp = cv2.imread(tp_path)
        tp = cv2.cvtColor(tp, cv2.COLOR_BGR2RGB)
        tp = cv2.resize(tp, (224, 224))
        
        test_files.remove(tp_path)
        
        tn_path = path + '/test/' + label + 'TN' + '/' + name + '_' + label + 'T18N.png'
        tn = cv2.imread(tn_path)
        tn = cv2.cvtColor(tn, cv2.COLOR_BGR2RGB)
        tn = cv2.resize(tn, (224, 224))
        
        test_files.remove(tn_path)
        
        tp = np.expand_dims(tp, axis = 0)
        tn = np.expand_dims(tn, axis = 0)
        tp = preprocess_input(tp)
        tn = preprocess_input(tn)

        prob_tp = model.predict(tp)
        prob_tn = model.predict(tn)
            
        pred_tp = np.argmax(prob_tp)
        pred_tn = np.argmax(prob_tn)
        preds_4.append(pred_tp)
        preds_4.append(pred_tn)
            
        #print("label: " + str(label))    
        #print('prediccion tp: ' + str(pred_tp))
        #print('prediccion tn: ' + str(pred_tn))

        if pred_tp == dict_labels['NTP'] and pred_tn == dict_labels['NTN']:
            pred = 'N'
        elif pred_tp == dict_labels['PTP'] and pred_tn == dict_labels['PTN']:
            pred = 'P'
        else:
            no_concuerda = no_concuerda + 1
            # prob_p = prob_tp[0][dict['PTP']] + prob_tp[0][dict['PTN']] + prob_tn[0][dict['PTP']] + prob_tn[0][dict['PTN']]
            # prob_n = prob_tp[0][dict['NTP']] + prob_tp[0][dict['NTN']] + prob_tn[0][dict['NTP']] + prob_tn[0][dict['NTN']]
            prob_p = max(prob_tp[0][dict_labels['PTP']], prob_tp[0][dict_labels['PTN']], prob_tn[0][dict_labels['PTP']], prob_tn[0][dict_labels['PTN']])
            prob_n = max(prob_tp[0][dict_labels['NTP']], prob_tp[0][dict_labels['NTN']], prob_tn[0][dict_labels['NTP']], prob_tn[0][dict_labels['NTN']])
            if prob_p >= prob_n:
                pred = 'P'
            else:
                pred = 'N'

        preds.append(pred)

    true_labels = np.array(true_labels)
    preds = np.array(preds)
    true_labels_4 = np.array(true_labels_4)
    preds_4 = np.array(preds_4)

    tabla_preds[:,1] = true_labels
    tabla_preds[:,2] = preds
    #np.savetxt(save_preds_file, tabla_preds, fmt = '%1s', delimiter = ',')

    # Calculate accuracy
    acc_4 = sum(true_labels_4 == preds_4)/len(true_labels_4)
    print('Accuracy 4 clases: ' + str(acc_4))
    print('Numero de veces no concuerda: ' + str(no_concuerda))
    acc = sum(true_labels == preds)/len(true_labels)
    results = classification_report(true_labels, preds, digits = 5, output_dict = True)

    #if results['N']['recall'] >= 0.73 and results['P']['recall'] >= 0.73:
    #    self._model.save(save_model_file)

    print(tabla_preds)
    
    print(results)
    print(acc)

In [60]:
dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 

print(len(test_files))

new_test_files = copy.deepcopy(test_files)

get_classification_report(new_test_files, dict_labels)

340
Accuracy 4 clases: 0.6441176470588236
Numero de veces no concuerda: 17
[['0638920510-120520' 'N' 'N']
 ['0787337378-030420' 'N' 'N']
 ['0708178611-250420' 'N' 'N']
 ['0696851031-200420' 'N' 'N']
 ['1705154118-130420' 'N' 'N']
 ['0818483876-230420' 'N' 'N']
 ['0806768094-240420' 'N' 'N']
 ['0720838525-130420' 'N' 'N']
 ['0761057351-200420' 'N' 'N']
 ['0915828834-010419' 'N' 'N']
 ['0725232423-170417' 'N' 'N']
 ['0700474989-100420' 'N' 'P']
 ['0692270813-310320' 'N' 'N']
 ['0816904392-231019' 'N' 'N']
 ['0697685029-030420' 'N' 'P']
 ['0736465730-260320' 'N' 'N']
 ['0702106108-240320' 'N' 'N']
 ['0597018126-150218' 'N' 'N']
 ['0694973776-040420' 'N' 'N']
 ['0384486981-110420' 'N' 'N']
 ['1553588887-290420' 'N' 'N']
 ['0943193342-220220' 'N' 'N']
 ['0690934031-140420' 'N' 'N']
 ['0818292304-140420' 'N' 'N']
 ['0930410964-130420' 'N' 'N']
 ['0730235906-280420' 'N' 'N']
 ['0714822000-280320' 'N' 'N']
 ['0711477823-220420' 'N' 'N']
 ['0380844734-100420' 'N' 'N']
 ['1530852895-260420' 'N' 